In [1]:
from datasets import load_dataset
import pandas as pd

# **Langkah 1:** Muat dataset berdasarkan ID repositori di Hugging Face
# ID repositori adalah: gsri-18/ISEAR-dataset-complete
dataset = load_dataset("gsri-18/ISEAR-dataset-complete")

# **Langkah 2:** Akses split yang Anda butuhkan (misalnya, 'train').
# Dataset ini memiliki satu split 'train'.
train_split = dataset["train"]

# **Langkah 3 (Opsional tapi Direkomendasikan):** Konversi ke Pandas DataFrame
# untuk kemudahan analisis dan manipulasi.
df = train_split.to_pandas()

# Tampilkan beberapa baris pertama untuk verifikasi
print("✅ Dataset Berhasil Dimuat dan Dikonversi ke Pandas DataFrame.")
print("--- Tinjauan Data ---")
df

# Sekarang Anda dapat bekerja dengan objek 'df' (DataFrame) atau 'train_split' (Dataset object).

/home/sorr/anaconda3/envs/preprocessing/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Dataset Berhasil Dimuat dan Dikonversi ke Pandas DataFrame.
--- Tinjauan Data ---


,emotion,content,Unnamed: 2
0,joy,On days when I feel close to my partner and ot...,None
1,fear,Every time I imagine that someone I love or I ...,None
2,anger,When I had been obviously unjustly treated and...,None
3,sadness,When I think about the short time that we live...,None
4,disgust,At a gathering I found myself involuntarily si...,None
...,...,...,...
7511,shame,Two years back someone invited me to be the tu...,None
7512,shame,I had taken the responsibility to do something...,None
7513,fear,I was at home and I heard a loud sound of spit...,None
7514,guilt,I did not do the homework that the teacher had...,None


In [2]:
print(df['emotion'].value_counts())

emotion
joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: count, dtype: int64


In [3]:
#drop unamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]   
df

,emotion,content
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...
...,...,...
7511,shame,Two years back someone invited me to be the tu...
7512,shame,I had taken the responsibility to do something...
7513,fear,I was at home and I heard a loud sound of spit...
7514,guilt,I did not do the homework that the teacher had...


In [4]:
keep_labels = ["joy", "fear", "anger", "sadness"]

df_cleaned = df[df['emotion'].isin(keep_labels)].reset_index(drop=True)
df_cleaned

,emotion,content
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,joy,After my girlfriend had taken her exam we went...
...,...,...
4324,anger,My roommate liked to listen to some meaningles...
4325,joy,I received a letter from a distant friend.
4326,fear,My parents were out and I was the eldest at ho...
4327,fear,I was at home and I heard a loud sound of spit...


In [5]:
import re

def preprocess_dataset(df):
    """Membersihkan dataset dari formatting issues"""

    def clean_content(text):
        if not isinstance(text, str):
            return text

        # Normalize newlines dan whitespace
        # Ganti semua newline dengan space
        text = re.sub(r'[\n\r]+', ' ', text)
        text = re.sub(r'\s+', ' ', text)      # Hapus multiple spaces
        text = text.strip()

        return text

    # Apply cleaning
    df['content'] = df['content'].apply(clean_content)

    # Hapus rows yang kosong
    df = df.dropna(subset=['content'])
    df = df[df['content'].str.strip() != '']

    return df

In [6]:
isear = preprocess_dataset(df_cleaned)
isear

,emotion,content
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,joy,After my girlfriend had taken her exam we went...
...,...,...
4324,anger,My roommate liked to listen to some meaningles...
4325,joy,I received a letter from a distant friend.
4326,fear,My parents were out and I was the eldest at ho...
4327,fear,I was at home and I heard a loud sound of spit...


In [ ]:

isear.to_csv('./data/isear_dataset.csv',
                  index=False,
                  encoding='utf-8-sig',
                  quoting=csv.QUOTE_ALL)

NameError: name 'csv' is not defined

In [10]:
import pandas as pd
from googletrans import Translator
import time
import logging
import requests

# Setup logging untuk monitor progress
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('translation_progress.log'),
        logging.StreamHandler()
    ]
)


class OptimizedTranslator:
    def __init__(self):
        self.translator = Translator()
        self.completed = 0
        self.total = 0

    def robust_translate(self, text, max_retries=5):
        """Translation dengan retry mechanism yang robust"""
        if not isinstance(text, str) or not text.strip():
            return text

        for attempt in range(max_retries):
            try:
                # Clean text sedikit
                clean_text = ' '.join(text.split())
                translated = self.translator.translate(
                    clean_text, dest='id', src='en')
                return translated.text

            except Exception as e:
                wait_time = (attempt + 1) * 10  # Exponential backoff
                logging.warning(
                    f"Attempt {attempt + 1} failed: {e}. Waiting {wait_time}s")
                time.sleep(wait_time)

                if attempt == max_retries - 1:
                    logging.error(
                        f"Failed to translate after {max_retries} attempts: {text[:100]}...")
                    return text  # Return original text if all retries fail

    def translate_dataset(self, input_file, output_file, start_from=0, batch_size=50):
        """Main translation function dengan progress saving"""

        # Load dataset
        df = pd.read_csv(input_file)
        self.total = len(df)

        # Initialize translated column
        if 'content_id' not in df.columns:
            df['content_id'] = df['content']

        logging.info(f"🚀 Starting translation for {self.total} records")
        logging.info(f"📁 Input: {input_file}")
        logging.info(f"💾 Output: {output_file}")

        for i in range(start_from, self.total):
            current_progress = i + 1

            # Progress logging
            if current_progress % batch_size == 0:
                progress_pct = (current_progress / self.total) * 100
                logging.info(
                    f"📊 Progress: {current_progress}/{self.total} ({progress_pct:.1f}%)")

            # Skip jika sudah diterjemahkan (untuk resume)
            if df.loc[i, 'content_id'] != df.loc[i, 'content']:
                self.completed += 1
                continue

            # Translate current record
            original_text = df.loc[i, 'content']
            translated_text = self.robust_translate(original_text)
            df.loc[i, 'content_id'] = translated_text
            self.completed += 1

            # Auto-save progress setiap 100 records
            if current_progress % 100 == 0:
                df.to_csv(
                    f"progress_backup_{current_progress}.csv", index=False)
                logging.info(f"💾 Backup saved at record {current_progress}")

            # Rate limiting - penting untuk hindari block!
            time.sleep(1)  # 1 detik antara request

        # Final save
        df.to_csv(output_file, index=False)
        logging.info(
            f"✅ Translation COMPLETED! {self.completed}/{self.total} records translated")
        logging.info(f"📁 Saved to: {output_file}")

        return df

    def resume_translation(self, progress_file, output_file):
        """Lanjutkan dari progress file"""
        df = pd.read_csv(progress_file)

        # Cari index terakhir yang sudah diterjemahkan
        last_translated = 0
        for i, row in df.iterrows():
            if row['content_id'] != row['content']:
                last_translated = i + 1

        logging.info(f"🔄 Resuming from record {last_translated}")
        return self.translate_dataset(progress_file, output_file, start_from=last_translated)

In [11]:
translator = OptimizedTranslator()

# OPTION 1: Start fresh translation
translator.translate_dataset(
    input_file='./data/isear_dataset.csv',
    output_file='./data/translated_dataset_final.csv',
    batch_size=50
)

2025-11-22 17:24:50,339 - INFO - 🚀 Starting translation for 4329 records
2025-11-22 17:24:50,340 - INFO - 📁 Input: ./data/isear_dataset.csv
2025-11-22 17:24:50,340 - INFO - 💾 Output: ./data/translated_dataset_final.csv
2025-11-22 17:24:50,341 - WARNING - Attempt 1 failed: 'coroutine' object has no attribute 'text'. Waiting 10s
/tmp/ipykernel_18479/375380642.py:33: RuntimeWarning: coroutine 'Translator.translate' was never awaited
  translated = self.translator.translate(
2025-11-22 17:25:00,342 - WARNING - Attempt 2 failed: 'coroutine' object has no attribute 'text'. Waiting 20s


KeyboardInterrupt: 